In [202]:
import pandas as pd                                         
import numpy as np                                              
from scipy.special import comb                                      
import math
from operator import mul
import neal 
import dimod
#from pyqubo import Array, Constraint, Placeholder, solve_qubo           
import itertools                                                        
import random                                                           
import matplotlib.pyplot as plt                                         
import timeit
from itertools import combinations

In [203]:
#=============                         
# 関数定義                          
#=============                                                        
# make_t_list_columns_num_samples         
# は正確検定の表の周辺和を計算しているので
# 次の関数に置き換え          
def calc_marginals(df):                   
    return np.array([                      
        sum(df['Y']),                     
        np.dot(df['Y'], df['LI']),      
        np.dot(df['Y'], df['SEX']),      
        np.dot(df['Y'], df['AOP']),      
    ])                                 

In [204]:
def make_Hamiltonian_(df, t1):
    t_list = calc_marginals(df)
    num_num =  tuple([str(n) for n in range(df.shape[0])])
    print('num_num:', num_num)
    combi_str_list = combinations(num_num, 2)
    combi_num_list = [tuple(map(int, i)) for i in (combi_str_list)]
   
    #式
    #(y0+y1+y2+...-r0)**2
    #or
    #(y0*1+y1*0+y2*1+...-r1)**2
    
    #Y
    lin_Y_1 = [1]*df.shape[0]#二乗の箇所その１
    lin_Y_2 = [-2*t_list[0]]*df.shape[0]#y*t_list[i]*2
    lin_Y = [l1 + l2 for (l1, l2) in zip(lin_Y_1, lin_Y_2)]
    quad_Y = [2]*len(combi_num_list)#y同士
    num_Y = t_list[0]**2#二乗の箇所その２
    
    #LI
    lin_LI_1 = [1 if df.iloc[i, 1]==1 else 0 for i in range(df.shape[0])]
    lin_LI_2 = [-2*t1 if df.iloc[i, 1]==1 else 0 for i in range(df.shape[0])]
    lin_LI = [l1 + l2 for (l1, l2) in zip(lin_LI_1, lin_LI_2)]
    quad_LI = [2 if df.iloc[each_conbi[0], 1]==1 and df.iloc[each_conbi[1], 1]==1 else 0 for each_conbi in (combi_num_list)]
    num_LI = t1**2#二乗の箇所その２
    
    #SEX
    lin_SEX_1 = [1 if df.iloc[i, 2]==1 else 0 for i in range(df.shape[0])]
    lin_SEX_2 = [-2*t1 if df.iloc[i, 2]==1 else 0 for i in range(df.shape[0])]
    lin_SEX = [l1 + l2 for (l1, l2) in zip(lin_SEX_1, lin_SEX_2)]
    quad_SEX = [2 if df.iloc[each_conbi[0], 2]==1 and df.iloc[each_conbi[1], 2]==1 else 0 for each_conbi in (combi_num_list)]
    num_SEX = t_list[2]**2#二乗の箇所その２
    
    #AOP
    lin_AOP_1 = [1 if df.iloc[i, 3]==1 else 0 for i in range(df.shape[0])]
    lin_AOP_2 = [-2*t1 if df.iloc[i, 3]==1 else 0 for i in range(df.shape[0])]
    lin_AOP = [l1 + l2 for (l1, l2) in zip(lin_AOP_1, lin_AOP_2)]
    quad_AOP = [2 if df.iloc[each_conbi[0], 3]==1 and df.iloc[each_conbi[1], 3]==1 else 0 for each_conbi in (combi_num_list)]
    num_AOP = t_list[3]**2#二乗の箇所その２

    
    ##統合作業
    #lin
    lin1 = [l1 + l2 for (l1, l2) in zip(lin_Y, lin_LI)]
    lin2 = [l1 + l2 for (l1, l2) in zip(lin_SEX, lin_AOP)]
    lin = [l1 + l2 for (l1, l2) in zip(lin1, lin2)]
    """lin_list = [l1 + l2 for (l1, l2) in zip(lin1, lin2)]
    
    lin = {}
    for i, each_lin in enumerate(lin_list):
        lin[i] = each_lin"""
    
    #quad
    quad1 = [q1 + q2 for (q1, q2) in zip(quad_Y, quad_LI)]
    quad2 = [q1 + q2 for (q1, q2) in zip(quad_SEX, quad_AOP)]
    quad_values = [q1 + q2 for (q1, q2) in zip(quad1, quad2)]#list
    
    quad = {}
    for i, each_combi in enumerate(combi_num_list):
        quad[each_combi] = quad_values[i]
    
    #num
    num = num_Y + num_LI + num_AOP + num_SEX
    
    print('lin:', lin)
    print('quad:', quad)
    print('num:', num)
    
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.BINARY)

In [205]:
def make_Hamiltonian__(df, t1):
    t_list = calc_marginals(df)
    
    same_num_combi_list = [(i, i) for i in range(df.shape[0])]
    print('same_num_combi_list:', same_num_combi_list)
    
    
    num_num_list = [str(n) for n in range(df.shape[0])]
    num_num = ''.join(num_num_list)
    print('num_num:', num_num)
    combi_str_list = itertools.combinations(num_num, 2)#str
    combi_num_list = [tuple(map(int, i)) for i in (combi_str_list)]
    print(combi_str_list)
   
    #式
    #(y0+y1+y2+...-r0)**2
    #or
    #(y0*1+y1*0+y2*1+...-r1)**2
    
    #Y
    lin_Y = [-2*t_list[0]]*df.shape[0]#y*t_list[i]*2
    quad_Y_1 = [1]*len(same_num_combi_list)#二乗の箇所その１
    quad_Y_2 = [2]*len(combi_num_list)#y同士
    num_Y = t_list[0]**2#二乗の箇所その２
    
    #LI
    lin_LI = [-2*t1 if df.iloc[i, 1]==1 else 0 for i in range(df.shape[0])]
    quad_LI_1 = [1 if df.iloc[each_same_num_combi[0], 1]==1 else 0 for each_same_num_combi in (same_num_combi_list)]
    quad_LI_2 = [2 if df.iloc[each_conbi[0], 1]==1 and df.iloc[each_conbi[1], 1]==1 else 0 for each_conbi in (combi_num_list)]
    num_LI = t1**2#二乗の箇所その２
    
    #SEX
    lin_SEX = [-2*t1 if df.iloc[i, 2]==1 else 0 for i in range(df.shape[0])]
    quad_SEX_1 = [1 if df.iloc[each_same_num_combi[0], 2]==1 else 0 for each_same_num_combi in (same_num_combi_list)]
    quad_SEX_2 = [2 if df.iloc[each_conbi[0], 2]==1 and df.iloc[each_conbi[1], 2]==1 else 0 for each_conbi in (combi_num_list)]
    num_SEX = t_list[2]**2#二乗の箇所その２
    
    #AOP
    lin_AOP = [-2*t1 if df.iloc[i, 3]==1 else 0 for i in range(df.shape[0])]
    quad_AOP_1 = [1 if df.iloc[each_same_num_combi[0], 3]==1 else 0 for each_same_num_combi in (same_num_combi_list)]
    quad_AOP_2 = [2 if df.iloc[each_conbi[0], 3]==1 and df.iloc[each_conbi[1], 3]==1 else 0 for each_conbi in (combi_num_list)]
    num_AOP = t_list[3]**2#二乗の箇所その２

    
    ##統合作業
    #lin
    lin1 = [l1 + l2 for (l1, l2) in zip(lin_Y, lin_LI)]
    lin2 = [l1 + l2 for (l1, l2) in zip(lin_SEX, lin_AOP)]
    lin_list = [l1 + l2 for (l1, l2) in zip(lin1, lin2)]
    lin = {}
    for i, li in enumerate(lin_list):
        lin[i]=li
    
    
    #quad_1
    quad1_1 = [q1 + q2 for (q1, q2) in zip(quad_Y_1, quad_LI_1)]
    quad1_2 = [q1 + q2 for (q1, q2) in zip(quad_SEX_1, quad_AOP_1)]
    quad_values1 = [q1 + q2 for (q1, q2) in zip(quad1_1, quad1_2)]#list
    
    quad = {}
    for i, each_same_num_combi in enumerate(same_num_combi_list):
        quad[each_same_num_combi] = quad_values1[i]
    print(quad)
    
    #quad_2
    quad2_1 = [q1 + q2 for (q1, q2) in zip(quad_Y_2, quad_LI_2)]
    quad2_2 = [q1 + q2 for (q1, q2) in zip(quad_SEX_2, quad_AOP_2)]
    quad_values2 = [q1 + q2 for (q1, q2) in zip(quad2_1, quad2_2)]#list
    

    for i, each_combi in enumerate(combi_num_list):
        quad[each_combi] = quad_values2[i]
        
    #num
    num = num_Y + num_LI + num_SEX + num_AOP
    
    print('lin:', lin)
    print('quad:', quad)
    print('num:', num)
    
    if dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY) == 0:
        print('no')
 
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

--------------

In [215]:
def make_Hamiltonian(df, t1):
    t_list = calc_marginals(df)
    
    same_num_combi_list = [(i, i) for i in range(df.shape[0])]
    #print('same_num_combi_list:', same_num_combi_list)
    
    
    num_num_list = [str(n) for n in range(df.shape[0])]
    num_num = ''.join(num_num_list)
    #print('num_num:', num_num)
    combi_str = itertools.combinations(num_num, 2)#str
    combi_num_list = [tuple(map(int, i)) for i in (combi_str)]
    #print('combi_num_list:', combi_num_list)
   
    #式
    #(y0+y1+y2+...-r0)**2
    #or
    #(y0*1+y1*0+y2*1+...-r1)**2
    
    #Y
    lin_Y = [-2*t_list[0]]*df.shape[0]#y*t_list[i]*2
    quad_Y_1 = [1]*len(same_num_combi_list)#二乗の箇所その１
    quad_Y_2 = [2]*len(combi_num_list)#y同士
    num_Y = t_list[0]**2#二乗の箇所その２
    
    #LI
    lin_LI = [-2*t1 if df.iloc[i, 1]==1 else 0 for i in range(df.shape[0])]
    quad_LI_1 = [1 if df.iloc[each_same_num_combi[0], 1]==1 else 0 for each_same_num_combi in (same_num_combi_list)]
    quad_LI_2 = [2 if df.iloc[each_conbi[0], 1]==1 and df.iloc[each_conbi[1], 1]==1 else 0 for each_conbi in (combi_num_list)]
    num_LI = t1**2#二乗の箇所その２
    
    #SEX
    lin_SEX = [-2*t_list[2] if df.iloc[i, 2]==1 else 0 for i in range(df.shape[0])]
    quad_SEX_1 = [1 if df.iloc[each_same_num_combi[0], 2]==1 else 0 for each_same_num_combi in (same_num_combi_list)]
    quad_SEX_2 = [2 if df.iloc[each_conbi[0], 2]==1 and df.iloc[each_conbi[1], 2]==1 else 0 for each_conbi in (combi_num_list)]
    num_SEX = t_list[2]**2#二乗の箇所その２
    
    #AOP
    lin_AOP = [-2*t_list[3] if df.iloc[i, 3]==1 else 0 for i in range(df.shape[0])]
    quad_AOP_1 = [1 if df.iloc[each_same_num_combi[0], 3]==1 else 0 for each_same_num_combi in (same_num_combi_list)]
    quad_AOP_2 = [2 if df.iloc[each_conbi[0], 3]==1 and df.iloc[each_conbi[1], 3]==1 else 0 for each_conbi in (combi_num_list)]
    num_AOP = t_list[3]**2#二乗の箇所その２

    
    ##統合作業
    #lin
    lin1 = [l1 + l2 for (l1, l2) in zip(lin_Y, lin_LI)]
    lin2 = [l1 + l2 for (l1, l2) in zip(lin_SEX, lin_AOP)]
    lin_list = [l1 + l2 for (l1, l2) in zip(lin1, lin2)]
    lin = {}
    for i, li in enumerate(lin_list):
        lin[i]=li
    
    
    #quad_1
    quad1_1 = [q1 + q2 for (q1, q2) in zip(quad_Y_1, quad_LI_1)]
    quad1_2 = [q1 + q2 for (q1, q2) in zip(quad_SEX_1, quad_AOP_1)]
    quad_values1 = [q1 + q2 for (q1, q2) in zip(quad1_1, quad1_2)]#list
    
    quad = {}
    for i, each_same_num_combi in enumerate(same_num_combi_list):
        quad[each_same_num_combi] = quad_values1[i]
    #print('quad_ongoing:',quad)
    
    #quad_2
    quad2_1 = [q1 + q2 for (q1, q2) in zip(quad_Y_2, quad_LI_2)]
    #print('quad2_1:', quad2_1)
    quad2_2 = [q1 + q2 for (q1, q2) in zip(quad_SEX_2, quad_AOP_2)]
    quad_values2 = [q1 + q2 for (q1, q2) in zip(quad2_1, quad2_2)]#list
    #print('quad_values2:', quad_values2)
    
    for i, each_num_combi in enumerate(combi_num_list):#str
        quad[each_num_combi] = quad_values2[i]
        #print('quad_going:', quad)
        
    #num
    num = num_Y + num_LI + num_SEX + num_AOP
    
    print('lin:', lin)
    print('quad:', quad)
    print('num:', num)
    
 
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

- [dimod.BinaryQuadraticModel](https://docs.ocean.dwavesys.com/en/latest/docs_dimod/reference/bqm_classes/binary_quadratic_model.html#dimod.BinaryQuadraticModel)
 - bqm = dimod.BinaryQuadraticModel({1: 1, 2: 1}, {(1, 2): 1}, 0.5, dimod.SPIN)
- [neal.sampler.SimulatedAnnealingSampler.sample()](https://docs.ocean.dwavesys.com/en/latest/docs_neal/reference/generated/neal.sampler.SimulatedAnnealingSampler.sample.html#neal.sampler.SimulatedAnnealingSampler.sample)

In [216]:
df = pd.read_csv('../../input/ost6.csv', sep=',', index_col=0)
print(df)
t1 = 2
bqm = make_Hamiltonian(df, t1)
#res = dimod.SimulatedAnnealingSampler().sample(bqm)
sa_sampler = neal.sampler.SimulatedAnnealingSampler()
res = sa_sampler.sample(bqm)
#res = neal.sampler.SimulatedAnnealingSampler.sample(bqm)
#res

'''
sa_sampler = SimulatedAnnealingSampler()
res = sa_sampler.sample(bqm)
'''

for y_info in list(res.record):
    print(y_info)
    
"""for s, e in res.data(['sample','energy']):
    print(s,'\t E = ', e)"""#わかりにくい

   Y  LI  SEX  AOP
0  1   1    0    0
1  0   1    1    1
2  0   1    1    0
3  0   1    1    1
4  0   1    0    1
5  1   1    0    1
lin: {0: -8, 1: -10, 2: -8, 3: -10, 4: -10, 5: -10}
quad: {(0, 0): 2, (1, 1): 4, (2, 2): 3, (3, 3): 4, (4, 4): 3, (5, 5): 3, (0, 1): 4, (0, 2): 4, (0, 3): 4, (0, 4): 4, (0, 5): 4, (1, 2): 6, (1, 3): 8, (1, 4): 6, (1, 5): 6, (2, 3): 6, (2, 4): 4, (2, 5): 4, (3, 4): 6, (3, 5): 6, (4, 5): 6}
num: 9
([1, 0, 0, 0, 0, 1], 0., 1)


"for s, e in res.data(['sample','energy']):\n    print(s,'\t E = ', e)"

In [217]:
def find_valid_y(df, num_reads):                                                        
    sa_sampler = neal.sampler.SimulatedAnnealingSampler()
    t_list = calc_marginals(df)
    
    valid_y_list = {}                                                                   
    valid_y_num = {}                                                                    
    for t1 in range(0, sum(df['LI'])+1):                                                                                                                                                                                                                                               
        bqm = make_Hamiltonian(df, t1)
        res = sa_sampler.sample(bqm)
        
        for s, e in res.data(['sample','energy']):
            print(s,'\t E = ', e)
        
        """res = neal.SimulatedAnnealingSampler().sample(bqm, num_reads=num_reads)
        print('res(t1=', t1, ')',res.record)"""                                                      
                                                                                        
        valid_y_list[t1] = []                                                           
        valid_y_num[t1] = 0                                                             
        for y_info in list(res.record):                                                 
            if y_info[1] == 0.:
                valid_y_num[t1] += 1                                        
                valid_y_list[t1].append(list(y_info[0]))                    
                print('energy0')
                
            '''if sum(y_info[0]) == t_list[0]:#int同士の比較  
                print('yes')
                y = pd.Series(y_info[0])                                                
                if np.dot([df['LI']], y) == t1:#int同士の比較                       
                    for j in range(2, len(t_list)):
                        v = df.to_numpy()[:, j]
                        if np.dot(v, y) - t_list[j] != 0:
                            break                                                       
                    else:                                                               
                        if all(list(y_info[0]) != p for p in valid_y_list[t1]):         
                            valid_y_num[t1] += 1                                        
                            valid_y_list[t1].append(list(y_info[0]))                    
                            print('perfect') '''                                           
    return valid_y_list, valid_y_num                                                                                                                                          

In [218]:
#==========
#テストコード
#==========
def test_find_valid_y():
    df = pd.read_csv('../../input/ost16.csv', sep=',', index_col=0)
    true_t1 = sum(df['Y'] * df['LI'])
    valid_y_list, valid_y_num = find_valid_y(df,  num_reads = 10)
    print(valid_y_list, valid_y_num)
    assert valid_y_num[true_t1] > 0  
    
test_find_valid_y()

lin: {0: -24, 1: -24, 2: -32, 3: -32, 4: -32, 5: -32, 6: -24, 7: -24, 8: -32, 9: -24, 10: -24, 11: -32, 12: -32, 13: -16, 14: -24, 15: -16}
quad: {(0, 0): 6, (1, 1): 6, (2, 2): 8, (3, 3): 8, (4, 4): 8, (5, 5): 8, (6, 6): 3, (7, 7): 3, (8, 8): 4, (9, 9): 3, (10, 10): 2, (11, 11): 4, (12, 12): 4, (13, 13): 1, (14, 14): 3, (15, 15): 2, (0, 1): 4, (0, 2): 6, (0, 3): 6, (0, 4): 6, (0, 5): 6, (0, 6): 6, (0, 7): 4, (0, 8): 6, (0, 9): 4, (1, 2): 6, (1, 3): 6, (1, 4): 6, (1, 5): 6, (1, 6): 4, (1, 7): 6, (1, 8): 6, (1, 9): 6, (1, 0): 4, (2, 3): 8, (2, 4): 8, (2, 5): 8, (2, 6): 6, (2, 7): 6, (2, 8): 8, (2, 9): 6, (2, 1): 6, (2, 0): 6, (3, 4): 8, (3, 5): 8, (3, 6): 6, (3, 7): 6, (3, 8): 8, (3, 9): 6, (3, 1): 6, (3, 0): 6, (3, 2): 8, (4, 5): 8, (4, 6): 6, (4, 7): 6, (4, 8): 8, (4, 9): 6, (4, 1): 6, (4, 0): 6, (4, 2): 8, (4, 3): 8, (5, 6): 6, (5, 7): 6, (5, 8): 8, (5, 9): 6, (5, 1): 6, (5, 0): 6, (5, 2): 8, (5, 3): 8, (5, 4): 8, (6, 7): 4, (6, 8): 6, (6, 9): 4, (6, 1): 4, (6, 0): 6, (6, 2): 6, (6, 3

AssertionError: 

In [214]:
def test_validity():
    df1 = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost16.csv', sep=',',index_col=0)
    new_y = np.array([1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]]) 

test_validity()

[8 6 4 4]
[8 7 5 4]


AssertionError: 